In [145]:
import pandas as pd

In [146]:
API_KEY = "cpk_c3cffbe8058f42b0a706de0ca0a7c9db.c3d10ff88d155b95be180e2f2383ffe2.781qC6zcKy0gzECGtZYDalrMakmkZti8"

In [147]:
import time
import requests, json

def invoke_chutes_stream(prompt, model="openai/gpt-oss-120b-TEE", show_reasoning=False):
    url = "https://llm.chutes.ai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
        "Accept": "text/event-stream",
    }

    body = {
        "model": model,
        "messages": [
            {"role": "system", "content": "Output ONLY the final answer text."},
            {"role": "user", "content": prompt},
        ],
        "stream": True,
        "max_tokens": 2048,
        "temperature": 0.0,
        "response_format": {
        "type": "json_schema",
            "json_schema": {
                "name": "answer_schema",
                "schema": {
                    "type": "object",
                    "properties": {
                        "answer": {
                            "type": "string",
                            "enum": ["A","B","C","D","E","F","G","H","I","J","K","L","M","N"],
                        }
                    },
                    "required": ["answer"],
                    "additionalProperties": False
                }
            }
        }
    }

    full = ""
    start = time.time()

    with requests.post(url, headers=headers, json=body, stream=True, timeout=120) as r:
        if not r.ok:
            print("STATUS:", r.status_code)
            print(r.text[:2000])
            r.raise_for_status()

        for raw in r.iter_lines(decode_unicode=True):
            if not raw or not raw.startswith("data:"):
                continue

            chunk = raw[len("data:"):].strip()
            if chunk == "[DONE]":
                break

            try:
                data = json.loads(chunk)
            except json.JSONDecodeError:
                continue

            choices = data.get("choices") or []
            if not choices:
                continue

            delta = (choices[0].get("delta") or {})

            # what you want (final)
            token = delta.get("content")

            # what you're actually receiving (reasoning stream)
            rtoken = delta.get("reasoning_content") or delta.get("reasoning")

            if token:
                #print(token, end="", flush=True)
                full += token
            elif show_reasoning and rtoken:
                # Only enable this if you WANT reasoning printed
                #print(rtoken, end="", flush=True)
                full += rtoken
    latency_s = time.time() - start

    obj = json.loads(full)
    return obj["answer"], obj, round(latency_s, 3)


In [148]:
import time
import random

results=[]
for i in range(250, 550):
    answer = invoke_chutes_stream(prompts.iloc[i,0], show_reasoning=False)
    print(i,answer)
    results.append((i, answer[0],answer[2]))
    time.sleep(random.uniform(1.5, 3.0))

250 ('H', {'answer': 'H'}, 5.298)
251 ('E', {'answer': 'E'}, 2.492)
252 ('D', {'answer': 'D'}, 2.556)
253 ('C', {'answer': 'C'}, 2.549)
254 ('E', {'answer': 'E'}, 2.332)
255 ('H', {'answer': 'H'}, 2.516)
256 ('C', {'answer': 'C'}, 2.504)
257 ('E', {'answer': 'E'}, 2.512)
258 ('D', {'answer': 'D'}, 2.301)
259 ('C', {'answer': 'C'}, 2.517)
260 ('D', {'answer': 'D'}, 2.204)
261 ('C', {'answer': 'C'}, 2.337)
262 ('C', {'answer': 'C'}, 2.324)
263 ('E', {'answer': 'E'}, 2.605)
264 ('D', {'answer': 'D'}, 3.467)
265 ('E', {'answer': 'E'}, 2.379)
266 ('G', {'answer': 'G'}, 4.865)
267 ('G', {'answer': 'G'}, 2.243)
268 ('F', {'answer': 'F'}, 1.829)
269 ('B', {'answer': 'B'}, 2.181)
270 ('C', {'answer': 'C'}, 2.001)
271 ('C', {'answer': 'C'}, 1.734)
272 ('G', {'answer': 'G'}, 2.469)
273 ('D', {'answer': 'D'}, 1.755)
274 ('E', {'answer': 'E'}, 1.964)
275 ('B', {'answer': 'B'}, 2.795)
276 ('A', {'answer': 'A'}, 2.634)
277 ('C', {'answer': 'C'}, 2.462)
278 ('E', {'answer': 'E'}, 2.282)
279 ('C', {'an

In [149]:
print(len(results))

300


In [150]:
results[0]

(250, 'H', 5.298)

In [151]:
import pandas as pd

df_results = pd.DataFrame(
    results,
    columns=["question_num", "pred_letter", "latency_s"]
)

df_results


,question_num,pred_letter,latency_s
0,250,H,5.298
1,251,E,2.492
2,252,D,2.556
3,253,C,2.549
4,254,E,2.332
...,...,...,...
295,545,A,4.383
296,546,E,2.263
297,547,D,2.375
298,548,C,3.362


In [152]:
df_results.to_csv('Old project/gpt-oss-250-549.csv', index=False)